# Building Fast Queries

We will imagine that we own an online laptop store and want to build a way to answer a few different business questions about our inventory.

The purpose of this is to practice building fast queries.

We'll use data from `laptops.csv`. A brief description of columns can be found below:

* **ID:** A unique identifier for the laptop.
* **Company:** The name of the company that produces the laptop.
* **Product:** The name of the laptop.
* **TypeName:** The type of laptop.
* **Inches:** The size of the screen in inches.
* **ScreenResolution:** The resolution of the screen.
* **CPU:** The laptop CPU.
* **RAM:** The amount of RAM in the laptop.
* **Memory:** The size of the hard drive.
* **GPU:** The graphics card name.
* **OpSys:** The name of the operating system.
* **Weight:** The laptop weight.
* **Price:** The price of the laptop.

### Import CSV

Let's import our data first.

In [ ]:
import csv

import random

import time

In [ ]:
with open("laptops.csv", "r") as f:
    reader = list(csv.reader(f))
    header = reader[0]
    rows = reader[1:]

In [101]:
# Header
for index, col in enumerate(header):
    print(f"{index:2} {col}")

 0 Id
 1 Company
 2 Product
 3 TypeName
 4 Inches
 5 ScreenResolution
 6 Cpu
 7 Ram
 8 Memory
 9 Gpu
10 OpSys
11 Weight
12 Price


In [100]:
# First 5 rows
for row in rows[:5]:
    print(row)
    print()

['6571244', 'Apple', 'MacBook Pro', 'Ultrabook', '13.3', 'IPS Panel Retina Display 2560x1600', 'Intel Core i5 2.3GHz', '8GB', '128GB SSD', 'Intel Iris Plus Graphics 640', 'macOS', '1.37kg', '1339']

['7287764', 'Apple', 'Macbook Air', 'Ultrabook', '13.3', '1440x900', 'Intel Core i5 1.8GHz', '8GB', '128GB Flash Storage', 'Intel HD Graphics 6000', 'macOS', '1.34kg', '898']

['3362737', 'HP', '250 G6', 'Notebook', '15.6', 'Full HD 1920x1080', 'Intel Core i5 7200U 2.5GHz', '8GB', '256GB SSD', 'Intel HD Graphics 620', 'No OS', '1.86kg', '575']

['9722156', 'Apple', 'MacBook Pro', 'Ultrabook', '15.4', 'IPS Panel Retina Display 2880x1800', 'Intel Core i7 2.7GHz', '16GB', '512GB SSD', 'AMD Radeon Pro 455', 'macOS', '1.83kg', '2537']

['8550527', 'Apple', 'MacBook Pro', 'Ultrabook', '13.3', 'IPS Panel Retina Display 2560x1600', 'Intel Core i5 3.1GHz', '8GB', '256GB SSD', 'Intel Iris Plus Graphics 650', 'macOS', '1.37kg', '1803']



### Inventory Class

Let's create a simple class to represent our inventory. We'll give it two attributes, `header` and `rows`

In [ ]:
class Inventory():
    def __init__(self, csv_filename):
        with open(csv_filename) as f:
            reader = list(csv.reader(f))
            self.header = reader[0]
            self.rows = reader[1:]
            
        for row in self.rows:
            row[-1] = int(row[-1])

In [ ]:
test = Inventory("laptops.csv")

In [ ]:
test.header

In [ ]:
len(test.rows)

### Finding a laptop from the ID

Let's update the `Inventory` class. We want to use it to find a laptop based on it's ID

In [ ]:
class Inventory():
    def __init__(self, csv_filename):
        with open(csv_filename) as f:
            reader = list(csv.reader(f))
            self.header = reader[0]
            self.rows = reader[1:]
        for row in self.rows:
            row[-1] = int(row[-1])
            
    def get_laptop_from_id(self, laptop_id):
        """Returns a row if ID is found"""
        for row in self.rows:
            identification = row[0]
            if identification == laptop_id:
                return row
        return None

In [ ]:
test = Inventory("laptops.csv")

In [ ]:
test.get_laptop_from_id('3362737')

In [ ]:
test.get_laptop_from_id('3362736')

This looks to be working.

### Improving ID Lookup

The previous algorithm has a time complexity of O(R), where R represents the number of rows. In order for this algo to work, it has to look through every single row.

There is a quicker way of dealing with this however.

One way would be to use a `set` to check if the ID exists. This would be constant time, rather than linear, which is an improvment. 

However, we also want to retrieve the row that corrosponds to the ID. For this, we'll instead use a `dictionary` object, which has the same fast lookup properties as `sets`, but also allow us to associate values with keys.

Let's try this out.

In [ ]:
class Inventory():
    def __init__(self, csv_filename):
        
        # Open file and assign header and rows
        with open(csv_filename) as f:
            reader = list(csv.reader(f))
            self.header = reader[0]
            self.rows = reader[1:]
            
        # Convert price to Int
        for row in self.rows:
            row[-1] = int(row[-1])
           
        # Store data to dictionary
        self.id_to_row = dict()
        for row in self.rows:
            self.id_to_row[row[0]] = row
            
    def get_laptop_from_id_fast(self, laptop_id):
        
        # Fast way to lookup ID
        for key, value in self.id_to_row.items():
            if key == laptop_id:
                return value
        return None
            
    def get_laptop_from_id(self, laptop_id):
        
        # Slow way to lookup ID
        for row in self.rows:
            identification = row[0]
            if identification == laptop_id:
                return row
        return None

In [ ]:
test = Inventory("laptops.csv")

In [ ]:
test.get_laptop_from_id_fast('3362737')

In [ ]:
test.get_laptop_from_id_fast('3362736')

This works. Unlike our previous algo which has a time complexity of O(R), this algoirthm has a time complexity of O(1). However, it does do this at the cost of memory (the dictionary) and there might be a little bit extra time for actually creating the dictionary on each instance.

### Comparing the Performance

Let's compare performance using the `time` module. We'll generate some random IDs first.

In [ ]:
ids = [random.randint(1000000, 9999999) for _ in range(100000)]

In [ ]:
test = Inventory("laptops.csv")

In [ ]:
total_time_no_dict = 0
for i in ids:
    start = time.time()
    test.get_laptop_from_id(i)
    end = time.time()
    runtime = end - start
    total_time_no_dict += runtime

In [ ]:
total_time_dict = 0
for i in ids:
    start = time.time()
    test.get_laptop_from_id_fast(i)
    end = time.time()
    runtime = end - start
    total_time_dict += runtime

In [ ]:
total_time_no_dict

In [ ]:
total_time_dict

Here we can see that using the dictionary, our times are faster.

### Two Laptop Promotion

The store in question sometimes offers giftcards - giftcards that can only be used once.

The problem here is that customers may feel cheated if they, for example, recieve a **£2,500** giftcard, and there's no way for them to spend all of that in a single transaction.

We therefore want to make sure there's a least one way to spend that full amount on a single transaction, whether there is a laptop worth that much, or a combination of laptops adding up to that much.

Let's write a function that, given a dollar amount, checks whether it is possible to spend precisely that amount by purchasing up to two laptops.

In [ ]:
class Inventory():
    def __init__(self, csv_filename):
        
        # Open file and assign header and rows
        with open(csv_filename) as f:
            reader = list(csv.reader(f))
            self.header = reader[0]
            self.rows = reader[1:]
            
        # Convert price to Int
        for row in self.rows:
            row[-1] = int(row[-1])
           
        # Store data to dictionary
        self.id_to_row = dict()
        for row in self.rows:
            self.id_to_row[row[0]] = row
            
    def get_laptop_from_id_fast(self, laptop_id):
        
        # Fast way to lookup ID
        for key, value in self.id_to_row.items():
            if key == laptop_id:
                return value
        return None
            
    def get_laptop_from_id(self, laptop_id):
        
        # Slow way to lookup ID
        for row in self.rows:
            identification = row[0]
            if identification == laptop_id:
                return row
        return None
    
    def check_promotion_dollar(self, dollars):
        for row in self.rows:
            price = row[-1]
            if price == dollars:
                return True
            for row2 in self.rows:
                if (row + row2) == price:
                    return True
        return False

In [ ]:
test = Inventory('laptops.csv')

In [ ]:
test.check_promotion_dollar(1000)

In [ ]:
test.check_promotion_dollar(442)

### Optimizing Laptop Promotion

Since we only care about whether or not there is a solution, we can store all laptops prices in a set when we initialize the inventory. Then we can check in constant time whether there is a laptop with a given price.

In [68]:
class Inventory():
    def __init__(self, csv_filename):
        
        # Open file and assign header and rows
        with open(csv_filename) as f:
            reader = list(csv.reader(f))
            self.header = reader[0]
            self.rows = reader[1:]
            
        # Convert price to Int
        for row in self.rows:
            row[-1] = int(row[-1])
           
        # Store data to dictionary
        self.id_to_row = dict()
        for row in self.rows:
            self.id_to_row[row[0]] = row
            
        # Create an empty set to store prices
        self.prices = set()
        for row in self.rows:
            self.prices.add(row[-1])
            
    def get_laptop_from_id_fast(self, laptop_id):
        
        # Fast way to lookup ID
        for key, value in self.id_to_row.items():
            if key == laptop_id:
                return value
        return None
            
    def get_laptop_from_id(self, laptop_id):
        
        # Slow way to lookup ID
        for row in self.rows:
            identification = row[0]
            if identification == laptop_id:
                return row
        return None
    
    def check_promotion_dollar(self, dollars):
        
        # Slow way to check
        for row in self.rows:
            price = row[-1]
            if price == dollars:
                return True
            for row2 in self.rows:
                if (row + row2) == price:
                    return True
        return False
    
    def check_promotion_dollar_fast(self, dollars):
        
        # Fast way to check
        if dollars in self.prices:
            return True
        for value in self.prices:
            if (dollars - value) in self.prices:
                return True
        return False

In [ ]:
test = Inventory("laptops.csv")

In [ ]:
test.check_promotion_dollar_fast(1000)

In [ ]:
test.check_promotion_dollar_fast(442)

### Comparing Promotion Functions

Now let's compare the peformance

In [ ]:
prices = [random.randint(100, 5001) for _ in range(100)]

In [69]:
test = Inventory("laptops.csv")

In [70]:
total_time_no_set = 0
for value in prices:
    start = time.time()
    test.check_promotion_dollar(value)
    end = time.time()
    runtime = end - start
    total_time_no_set += runtime

In [71]:
total_time_set = 0
for value in prices:
    start = time.time()
    test.check_promotion_dollar_fast(value)
    end = time.time()
    runtime = end - start
    total_time_set += runtime

In [72]:
total_time_no_set

23.42894434928894

In [73]:
total_time_set

0.0005838871002197266

Wow! That's quite a difference there.

### Finding Laptops Within A Budget

Next up, we want to answer one question:

Given a budget of D dollars, find all laptops whose price is at most D.

In [81]:
class Inventory():
    def __init__(self, csv_filename):
        
        # Open file and assign header and rows
        with open(csv_filename) as f:
            reader = list(csv.reader(f))
            self.header = reader[0]
            self.rows = reader[1:]
            
        # Convert price to Int
        for row in self.rows:
            row[-1] = int(row[-1])
           
        # Store data to dictionary
        self.id_to_row = dict()
        for row in self.rows:
            self.id_to_row[row[0]] = row
            
        # Create an empty set to store prices
        self.prices = set()
        for row in self.rows:
            self.prices.add(row[-1])
            
        # Sort rows by price
        self.rows_by_price = sorted(self.rows, key = lambda x: x[-1])
            
    def get_laptop_from_id_fast(self, laptop_id):
        
        # Fast way to lookup ID
        for key, value in self.id_to_row.items():
            if key == laptop_id:
                return value
        return None
            
    def get_laptop_from_id(self, laptop_id):
        
        # Slow way to lookup ID
        for row in self.rows:
            identification = row[0]
            if identification == laptop_id:
                return row
        return None
    
    def check_promotion_dollar(self, dollars):
        
        # Slow way to check
        for row in self.rows:
            price = row[-1]
            if price == dollars:
                return True
            for row2 in self.rows:
                if (row + row2) == price:
                    return True
        return False
    
    def check_promotion_dollar_fast(self, dollars):
        
        # Fast way to check
        if dollars in self.prices:
            return True
        for value in self.prices:
            if (dollars - value) in self.prices:
                return True
        return False
    
    def find_first_laptop_more_expensive(self, target_price):
        # Return index of the first row in self.rows_by_price whose price is higher than price. 
        range_start = 0
        range_end = len(self.rows_by_price) - 1
        
        while range_start < range_end:
            
            range_middle = (range_start + range_end) // 2
            value = self.rows_by_price[range_middle][-1]
            
            if value <= target_price:
                range_start = range_middle + 1
            else:
                range_end = range_middle

        value = self.rows_by_price[range_start][-1]
        
        if target_price >= value:
            return -1
        
        return range_start


In [82]:
test = Inventory("laptops.csv")

In [83]:
test.find_first_laptop_more_expensive(1000)

683

In [84]:
test.find_first_laptop_more_expensive(10000)

-1

### Min and Max Price

Imagine that we extend our budget query to take as input a range of prices, min_price and max_price, rather than a single price. Write a query that finds all laptops whose price is in the given range.

In [93]:
class Inventory():
    def __init__(self, csv_filename):
        
        # Open file and assign header and rows
        with open(csv_filename) as f:
            reader = list(csv.reader(f))
            self.header = reader[0]
            self.rows = reader[1:]
            
        # Convert price to Int
        for row in self.rows:
            row[-1] = int(row[-1])
           
        # Store data to dictionary
        self.id_to_row = dict()
        for row in self.rows:
            self.id_to_row[row[0]] = row
            
        # Create an empty set to store prices
        self.prices = set()
        for row in self.rows:
            self.prices.add(row[-1])
            
        # Sort rows by price
        self.rows_by_price = sorted(self.rows, key = lambda x: x[-1])
            
    def get_laptop_from_id_fast(self, laptop_id):
        
        # Fast way to lookup ID
        for key, value in self.id_to_row.items():
            if key == laptop_id:
                return value
        return None
            
    def get_laptop_from_id(self, laptop_id):
        
        # Slow way to lookup ID
        for row in self.rows:
            identification = row[0]
            if identification == laptop_id:
                return row
        return None
    
    def check_promotion_dollar(self, dollars):
        
        # Slow way to check
        for row in self.rows:
            price = row[-1]
            if price == dollars:
                return True
            for row2 in self.rows:
                if (row + row2) == price:
                    return True
        return False
    
    def check_promotion_dollar_fast(self, dollars):
        
        # Fast way to check
        if dollars in self.prices:
            return True
        for value in self.prices:
            if (dollars - value) in self.prices:
                return True
        return False
    
    def find_first_laptop_more_expensive(self, target_price):
        # Return index of the first row in self.rows_by_price whose price is higher than price. 
        range_start = 0
        range_end = len(self.rows_by_price) - 1
        
        while range_start < range_end:
            
            range_middle = (range_start + range_end) // 2
            value = self.rows_by_price[range_middle][-1]
            
            if value <= target_price:
                range_start = range_middle + 1
            else:
                range_end = range_middle

        value = self.rows_by_price[range_start][-1]
        
        if target_price >= value:
            return -1
        
        return range_start

    def find_laptops_in_range(self, min_price, max_price):
        for row in self.rows:
            price = row[-1]
            if price > min_price and price < max_price:
                print(row)

In [99]:
test = Inventory("laptops.csv")
test.find_laptops_in_range(500, 520)

['2751361', 'Lenovo', 'IdeaPad 320-15IKBN', 'Notebook', '15.6', 'Full HD 1920x1080', 'Intel Core i5 7200U 2.5GHz', '8GB', '2TB HDD', 'Intel HD Graphics 620', 'No OS', '2.2kg', 519]
['6382974', 'Asus', 'VivoBook S14', 'Notebook', '14', '1366x768', 'Intel Core i3 7100U 2.4GHz', '4GB', '128GB SSD', 'Intel HD Graphics 620', 'Windows 10', '1.3kg', 509]
['1998281', 'Lenovo', 'IdeaPad 320-15AST', 'Notebook', '17.3', '1600x900', 'AMD A6-Series 9220 2.5GHz', '8GB', '1TB HDD', 'AMD Radeon R4', 'Windows 10', '2.8kg', 519]
['8866576', 'Asus', 'VivoBook Max', 'Notebook', '15.6', 'Full HD 1920x1080', 'Intel Core i5 7200U 2.5GHz', '4GB', '1TB HDD', 'Nvidia GeForce 920', 'Linux', '2.1kg', 519]
['2878339', 'Dell', 'Inspiron 5578', '2 in 1 Convertible', '15', 'Full HD / Touchscreen 1920x1080', 'Intel Core i3 7100U 2.4GHz', '4GB', '500GB HDD', 'Intel HD Graphics 620', 'Windows 10', '2.08kg', 509]


### Cheapest Laptop with Characteristics (RAM & Storage)

Let's allow customers to find the cheapest laptop that matches desired characteristics. We'll only focus on RAM (index 7) and Storage here (index 8). We'll need to convert these to integers first.

In [124]:
class Inventory():
    def __init__(self, csv_filename):
        
        # Open file and assign header and rows
        with open(csv_filename) as f:
            reader = list(csv.reader(f))
            self.header = reader[0]
            self.rows = reader[1:]
            
        # Convert price to Int, along with ram & memory
        for row in self.rows:
            row[-1] = int(row[-1])
            row[7] = int(row[7][:-2])
            row[8] = int(''.join(i for i in row[8] if i.isdigit()))
           
        # Store data to dictionary
        self.id_to_row = dict()
        for row in self.rows:
            self.id_to_row[row[0]] = row
            
        # Create an empty set to store prices
        self.prices = set()
        for row in self.rows:
            self.prices.add(row[-1])
            
        # Sort rows by price
        self.rows_by_price = sorted(self.rows, key = lambda x: x[-1])
            
    def get_laptop_from_id_fast(self, laptop_id):
        
        # Fast way to lookup ID
        for key, value in self.id_to_row.items():
            if key == laptop_id:
                return value
        return None
            
    def get_laptop_from_id(self, laptop_id):
        
        # Slow way to lookup ID
        for row in self.rows:
            identification = row[0]
            if identification == laptop_id:
                return row
        return None
    
    def check_promotion_dollar(self, dollars):
        
        # Slow way to check
        for row in self.rows:
            price = row[-1]
            if price == dollars:
                return True
            for row2 in self.rows:
                if (row + row2) == price:
                    return True
        return False
    
    def check_promotion_dollar_fast(self, dollars):
        
        # Fast way to check
        if dollars in self.prices:
            return True
        for value in self.prices:
            if (dollars - value) in self.prices:
                return True
        return False
    
    def find_first_laptop_more_expensive(self, target_price):
        # Return index of the first row in self.rows_by_price whose price is higher than price. 
        range_start = 0
        range_end = len(self.rows_by_price) - 1
        
        while range_start < range_end:
            
            range_middle = (range_start + range_end) // 2
            value = self.rows_by_price[range_middle][-1]
            
            if value <= target_price:
                range_start = range_middle + 1
            else:
                range_end = range_middle

        value = self.rows_by_price[range_start][-1]
        
        if target_price >= value:
            return -1
        
        return range_start

    def find_laptops_in_range(self, min_price, max_price):
        for row in self.rows:
            price = row[-1]
            if price > min_price and price < max_price:
                print(row)
                
    def find_cheap_laptop_for_characteristics(self, target_ram, target_memory):
        
        for row in self.rows_by_price:
            ram = row[7]
            memory = row[8]
            if ram == target_ram and memory == target_memory:
                return row
        return -1
            

In [125]:
test = Inventory("laptops.csv")
test.find_cheap_laptop_for_characteristics(8, 2)

['2946920',
 'HP',
 '15-AY023na (N3710/8GB/2TB/W10)',
 'Notebook',
 '15.6',
 '1366x768',
 'Intel Pentium Quad Core N3710 1.6GHz',
 8,
 2,
 'Intel HD Graphics 405',
 'Windows 10',
 '2.04kg',
 389]